In [2]:
import pandas as pd
import resource 
%load_ext memory_profiler

file = 'block_total.csv'
total_lines = 17_703_908
#LCLid,tstp,energy(kWh/hh)
pred = (lambda x: (x['energy(kWh/hh)'].astype('float')) < 0.6)

**Ideas**

Set max memory usage and have chunksize dynamically adjust

parse out columns and pickle so can do fast query on column to get list of rows for selection

however, then requires two file reads and still have to seek through all data


In [2]:
%%time
%%memit

SAMPLE_SIZE = 100
MAX_MEM_MB = 250
MAX_MEM_BYTES = 1024*1024*MAX_MEM_MB

iterator = pd.read_csv(file, iterator=True)

df_head = iterator.get_chunk(size=SAMPLE_SIZE)

avg_line_bytes = 1.1 * df_head.memory_usage(deep=True).sum()/SAMPLE_SIZE
print(f'avg line bytes {avg_line_bytes}')

total_mem_estimate = total_lines * avg_line_bytes
print(f'MEM estimate {total_mem_estimate}')

df_read_size = 0
lines_read = SAMPLE_SIZE - 1

dfs = [df_head]
chunk_count = 0

iterator = pd.read_csv(file, iterator=True)


while lines_read < total_lines:
    chunk_count += 1
    lines_to_read = min(int((MAX_MEM_BYTES - df_read_size) / avg_line_bytes), total_lines-lines_read)
    print(f"ltr {lines_to_read}")
    df_chunk = iterator.get_chunk(size=lines_to_read)
    
    indices = df_chunk[pred].index
    df_chunk.drop(indices, inplace=True)
    
#     chunk_mem = df_chunk.memory_usage(deep=True).sum() #actual
    chunk_mem = (lines_to_read - len(indices))  * avg_line_bytes #fast estimate
    
    dfs.append(df_chunk)
    
    lines_read += lines_to_read
    df_read_size += chunk_mem


df_total = pd.concat(dfs)

print(f'Total Chunks: {chunk_count}')
# chunk_mem = df_total.memory_usage(deep=True).sum()
# print(f'Total Mem Usage  {chunk_mem/(1024*1024)} MB')
print(df_total.shape)

avg line bytes 175.20800000000003
MEM estimate 3101866312.8640003
ltr 1496187
ltr 1127782
ltr 932692
ltr 826136
ltr 731836
ltr 692819
ltr 670989
ltr 644524
ltr 610125
ltr 594049
ltr 575115
ltr 545083
ltr 528607
ltr 500047
ltr 480376
ltr 453198
ltr 411902
ltr 374286
ltr 368200
ltr 352163
ltr 336456
ltr 328702
ltr 324215
ltr 320054
ltr 295931
ltr 288141
ltr 278728
ltr 264997
ltr 251829
ltr 242174
ltr 229764
ltr 218553
ltr 211013
ltr 206386
ltr 201413
ltr 191528
ltr 189142
ltr 181505
ltr 175871
ltr 51291
Total Chunks: 40
(1324834, 3)
peak memory: 249.11 MiB, increment: 175.82 MiB
CPU times: user 13.1 s, sys: 1.72 s, total: 14.9 s
Wall time: 15.1 s


In [5]:
%%time
%%memit

CHUNKSIZE = 50_000
dfs = []

for i, df in enumerate(pd.read_csv(file, iterator=True, chunksize=CHUNKSIZE)):
    indices = df[pred].index
#     chunk_mem = df.memory_usage(deep=True).sum()
    df.drop(indices, inplace=True)
#     print(f'chunkmem {chunk_mem/(1024*1024)} MB')
    dfs.append(df)

df_total = pd.concat(dfs)

# chunk_mem = df_total.memory_usage(deep=True).sum()
print(f'{i} chunks of size {CHUNKSIZE} read')
# print(f'Total Current Mem Usage  {chunk_mem/(1024*1024)} MB')

print(df_total.shape)

354 chunks of size 50000 read
(1324734, 3)
peak memory: 377.87 MiB, increment: 44.68 MiB
CPU times: user 17.4 s, sys: 2.93 s, total: 20.3 s
Wall time: 21 s


In [20]:
ls = list(indices)

In [21]:
%%time
%%memit
# print(list(indices))
df = pd.read_csv(file, skiprows=ls)

# indices = df[pred].index

# chunk_mem = df.memory_usage(deep=True).sum()
# print(chunk_mem)

# df.drop(indices, inplace=True)
print(df.shape)


(1324734, 3)
peak memory: 1950.93 MiB, increment: 337.46 MiB
CPU times: user 6.94 s, sys: 1.5 s, total: 8.44 s
Wall time: 8.73 s


In [4]:
%%time
%%memit

CHUNKSIZE = 600_000
dfs = []

iterator = pd.read_csv(file, iterator=True, chunksize=CHUNKSIZE)


#     indices = df[pred].index
# #     chunk_mem = df.memory_usage(deep=True).sum()
#     df.drop(indices, inplace=True)
# #     print(f'chunkmem {chunk_mem/(1024*1024)} MB')
#     dfs.append(df)

# df_total = pd.concat(dfs)

# chunk_mem = df_total.memory_usage(deep=True).sum()
# print(f'{i} chunks of size {CHUNKSIZE} read')
# print(f'Total Current Mem Usage  {chunk_mem/(1024*1024)} MB')

# print(df_total.shape)


peak memory: 78.72 MiB, increment: 6.03 MiB
CPU times: user 53 ms, sys: 34.9 ms, total: 87.8 ms
Wall time: 223 ms
